In [ ]:
# installs
! pip install scikit-learn
! pip install umap-learn
! pip install scikit-plot


In [ ]:
#imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm

#scipy
from scipy.cluster.hierarchy import linkage, dendrogram
from scipy.cluster.hierarchy import fcluster
from scipy.spatial.distance import pdist, squareform

#sklearn
from sklearn import metrics 
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.manifold import MDS
from sklearn.manifold import TSNE
from sklearn.datasets import load_iris
from sklearn.cluster import KMeans, DBSCAN
from sklearn.neighbors import NearestNeighbors

import scikitplot as skplot



In [ ]:
# auth into GCP Big Query

# COLAB Only
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

# for non-Colab
# see resources, as long as token with env var setup properly, below should work

In [ ]:
# get the data
SQL = "SELECT * from `questrom.datasets.mtcars`"
YOUR_BILLING_PROJECT = "ba-775pd"

df = pd.read_gbq(SQL, YOUR_BILLING_PROJECT)

# Examples

In [ ]:
# lets drop the model column and use it as the index
cars.index = cars.model
cars.drop(columns="model", inplace=True)


In [ ]:
# keep just the continous variables
cars2 = pd.concat((cars.loc[:, "mpg"], cars.loc[:, "disp":"qsec"]), axis=1)

# or
# drop the unnessary columns, I identified in the step ahead
stocks.drop(columns=['Quarter end', 'ticker', 'quarter_end', 'Split factor','Shares split adjusted'], inplace=True)

In [ ]:
# replace missing values with the mean and lower case column names
stocks = stocks.fillna(stocks.mean())
stocks.columns = stocks.columns.str.lower()

In [ ]:
# double checking if there are any missing values 
print(stocks.isna().sum().sum())

# Scaling the Data
only if the data varies a lot

In [ ]:
# first, I am going to scale the data given the varying units of measurement
sc = StandardScaler() #standardize features
nl = Normalizer() #rescales each sample
sm = sc.fit_transform(stocks)

sm = pd.DataFrame(sm, columns=stocks.columns)

# confirm the changes
sm.head(3)

# Hierarchical Clustering

In [ ]:
# Hierarchical Clustering - first attempt
# going to do euclidean, cosine, jaccard, cityblock distance
diste = pdist(sm.values)
distc = pdist(sm.values, metric="cosine")
distj = pdist(sm.values, metric="jaccard")
distm = pdist(sm.values, metric="cityblock")

# put all on the same linkage to compare
hclust_e = linkage(diste)
hclust_c = linkage(distc)
hclust_j = linkage(distj)
hclust_m = linkage(distm)

# plots
LINKS = [hclust_e, hclust_c, hclust_j,hclust_m]
TITLE = ['Euclidean', 'Cosine', 'Jaccard', 'Manhattan']

plt.figure(figsize=(15, 5))

# loop and build our plot
for i, m in enumerate(LINKS):
  plt.subplot(1, 4, i+1)
  plt.title(TITLE[i])
  dendrogram(m,
             leaf_rotation=90,
             orientation="left")
  
plt.show()


In [ ]:
#chose the one with the best clusterts - more "clusters" visible
METHODS = ['single', 'complete', 'average', 'ward']
plt.figure(figsize=(20,5))


# loop and build our plot for the different methods
for i, m in enumerate(METHODS):
  plt.subplot(1, 4, i+1)
  plt.title(m)
  dendrogram(linkage(distc, method=m), 
             leaf_rotation=90)

In [ ]:
# Example: using cosine + complete
# the labels with 7 clusters
labs = fcluster(linkage(distc, method="complete"), 7, criterion="maxclust")

# confirm
np.unique(labs)

# add a cluster column to the stocks data set
stocks['cluster'] = labs
print(stocks.head(3))

#let see if the data in the cluster is evenly distributed
print(stocks.cluster.value_counts(dropna=False, sort=False))

In [ ]:
# cluster solution
clus_profile = stocks.groupby("cluster").mean()

clus_profile.T

# heatmap plot of the clusters with normalized data
scp = StandardScaler()
cluster_scaled = scp.fit_transform(clus_profile)

cluster_scaled = pd.DataFrame(cluster_scaled, 
                              index=clus_profile.index, 
                              columns=clus_profile.columns)

sns.heatmap(cluster_scaled, cmap="Blues", center=0)
plt.show()

# KMeans


In [ ]:
# another method: KMeans 
xs = sc.fit_transform(stocks)
X = pd.DataFrame(xs, index=stocks.index, columns=stocks.columns)

# Kmeans for 2 to 30 clusters
KS = range(2, 30)

# storage
inertia = []
silo = []

for k in KS:
  km = KMeans(k)
  km.fit(X)
  labs = km.predict(X)
  inertia.append(km.inertia_)
  silo.append(silhouette_score(X, labs))

print(silo)

In [ ]:
#plot 
plt.figure(figsize=(15,5))


plt.subplot(1, 2, 1)
plt.title("Inertia")
sns.lineplot(KS, inertia)

plt.subplot(1, 2, 2)
plt.title("Silohouette Score")
sns.lineplot(KS, silo)

plt.show()

In [ ]:
for i, s in enumerate(silo[:10]):
  print(i+2,s)

In [ ]:
# looks like 5 is a good point
# get the model
k5 = KMeans(5)
k5_labs = k5.fit_predict(X)

# metrics
k5_silo = silhouette_score(X, k5_labs)
k5_ssamps = silhouette_samples(X, k5_labs)
np.unique(k5_labs)

In [ ]:
# lets compare via silo

skplot.metrics.plot_silhouette(X, labs, title="HClust", figsize=(15,5))
plt.show()

In [ ]:
skplot.metrics.plot_silhouette(X, k5_labs, title="KMeans - 5", figsize=(15,5))
plt.show()

# PCA

In [ ]:
# Let try another method -PCA

pca = PCA()
pcs = pca.fit_transform(stocks)

In [ ]:
# shape confirmation (rows/features) are identical SHAPES
pcs.shape == stocks.shape

In [ ]:
# first, lets get the explained variance
# elbow plot

varexp = pca.explained_variance_ratio_
sns.lineplot(range(1, len(varexp)+1), varexp)

In [ ]:
# cumulative variance

plt.title("Cumulative Explained Variance")
plt.plot(range(1, len(varexp)+1), np.cumsum(varexp))
plt.axhline(.987)

In [ ]:
# quick function to construct the barplot easily
def ev_plot(ev):
  y = list(ev)
  x = list(range(1,len(ev)+1))
  return x, y

In [ ]:
# another approach for selection is to use explained variance > 1

x, y = ev_plot(pca.explained_variance_)
sns.barplot(x, y)